In [ ]:
4. Writing Data to Delta Format with Partitioning

In [ ]:
spark.sql("SHOW DATABASES").show()

In [ ]:
USE eea_curated;

In [ ]:
Convert the Table to Delta Format

In [ ]:
The CO2 emissions table on the curated layer

In [ ]:

# Specify the S3 bucket and path
s3_bucket = "databricks-workspace-stack-10fab-bucket"
s3_path = "unity-catalog/1803637943354536"

df_path = f"s3a://{s3_bucket}/{s3_path}/eea_curated/co2_emissions/"


In [ ]:
%sql
df_path = f"s3a://{s3_bucket}/{s3_path}/eea_curated/co2_emissions/"
CONVERT TO DELTA parquet.`s3://your-bucket-name/eea_curated/co2_emissions/` PARTITIONED BY (year)

In [ ]:
The highest emissions table on the serving layer

In [ ]:
%sql
df_path = f"s3a://{s3_bucket}/{s3_path}/eea_serving/highest_emissions/"
CONVERT TO DELTA parquet.`s3://your-bucket-name/eea_serving/highest_emissions/` PARTITIONED BY (year, MS)


In [ ]:
The emissions change YoY table on the serving layer

In [ ]:
%sql
df_path = f"s3a://{s3_bucket}/{s3_path}/eea_serving/emissions_diff_yoy/"
CONVERT TO DELTA parquet.`s3://your-bucket-name/eea_serving/emissions_diff_yoy/` PARTITIONED BY (year)

In [ ]:
5. operations were executed

 Here's an overview of the key operations and processes involved:
1. Reading and Analyzing Source Data

2. Data Conversion

3. Transaction Log Update

4. Data Skipping and Indexing (if Z-Ordering used)

5. Checkpointing

6. Optimizations for Read Performance


In [ ]:
6.1 Optimisarion of the newly created Delta tables

In [ ]:
%sql
OPTIMIZE eea_curated.co2_emissions_delta ZORDER BY (MS)

In [ ]:
%sql
OPTIMIZE eea_serving.highest_emissions

In [ ]:
%sql
OPTIMIZE eea_serving/emissions_diff_yoya ZORDER BY (MS)

In [ ]:
# Define the path where the Delta table will be stored
delta_table_path = "/path/to/delta/highest_emissions"

# Write the DataFrame to Delta format, partitioned by 'year' and 'MS'
(df_highest_emissions
    .write
    .format("delta")
    .partitionBy("year", "MS")
    .mode("overwrite")
    .save(delta_table_path))

In [ ]:
CONVERT TO DELTA parquet.`/path/to/non-delta/highest_emissions`
PARTITIONED BY (year INT, MS STRING)

In [ ]:
from delta.tables import DeltaTable

# Assuming you have the DeltaTable object
delta_table = DeltaTable.forPath(spark, "/path/to/delta/table")

# Optimize the table and apply Z-Ordering on the 'MS' column
delta_table.optimize().zorderBy("MS").execute()

In [ ]:
# Define the path for the Delta table
delta_table_path = "dbfs:/path/to/datalake/delta_table"

# Write the DataFrame to Delta format
df.write.format("delta").mode("overwrite").save(delta_table_path)

In [ ]:
df.write.format("delta").partitionBy("partitionColumn").mode("overwrite").save(delta_table_path)

In [ ]:
df.write.format("delta").mode("overwrite").saveAsTable("database_name.table_name")
